In [ ]:
import sys
sys.path.append('..')
from lib.download import *
from tqdm import tqdm
import numpy as np
import pandas as pd
import snscrape.modules.twitter as sntwitter
import pandas as pd
import time
import torch.nn.functional as F

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By

import concurrent.futures
import queue

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification, pipeline
from scipy.special import expit

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
topictokenizer = AutoTokenizer.from_pretrained("cardiffnlp/tweet-topic-21-multi")
topicmodel = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/tweet-topic-21-multi")
topic_mapping = topicmodel.config.id2label # Links between predicted classes and corresponding topics
topicmodel.to(device)  # CUDA

###FLS
flsmodel = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3)
flstokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')
nlp = pipeline("text-classification", model=flsmodel, tokenizer=flstokenizer)
flsmodel.to(device)  # CUDA

def pred_topic(tweet):
    inputs = topictokenizer(tweet, return_tensors="pt")
    outputs = topicmodel(**inputs)
    scores = outputs[0][0].detach().numpy()
    scores = expit(scores)
    predictions = (scores >= 0.5) * 1                                    
    return [topic_mapping[predictions.argmax()]]

def pred_fls(tweet):  
    return list(nlp(tweet)[0].values())

In [ ]:
torch.cuda.is_available()

In [ ]:
#### retrieve preselected twitter usernames ####

user_list = pd.read_excel('data/tweets/usernames.xlsx')['Usernames'].tolist()
user_list = [user.lstrip() for user in user_list]

df = pd.read_csv(f'data/tweets/{"@BTC_Archive"}.csv')
df.tail()

In [ ]:
!pip uninstall torch 

In [ ]:
def add_tweet_db_attributes(username_list):
    for user in user_list:
        attributes = []
        tweets = pd.read_csv(f'data/tweets/{user}.csv')
        for tweet in tqdm(df["Tweet"]):
            attributes.append(pred_topic(tweet)+pred_fls(tweet))
        attributes_df = pd.DataFrame(attributes,columns=['topic','fls','fls_score'])
        pd.concat([tweets,attributes_df], axis=1).to_csv(f'data/tweets/{user}_attributes.csv')

In [4]:
import pandas as pd
pd.read_csv('data/tweets/@elonmusk_attributes.csv').iloc[:,3:].to_excel('exmple.xlsx')

In [ ]:
add_tweet_db_attributes(user_list[:2])

In [ ]:
autoformer  tres bonnes performances et comprehensible aussi
sionon timesnet

In [ ]:
avantage tft est quantiles pr stoploss + machineln

In [ ]:
def hourly_summarizer()
gpt? fichier brut = > output direct par  heure voir colab gpt

In [ ]:
def get_tweet_context-
avec gpt aussi, dire neutre si pas assez a summarize, volir ds sntweeter por la cconv

In [ ]:
peut meme aller encore plus loin en demandant tout a chatgpt en une fois (fls...............)

In [ ]:
get  attributes sur le summarizer

In [ ]:
backtrader

In [ ]:
pca pr analyse des tweets les mieux

dans un premier temps nelever tous les arobases

bcp de tweets sont analyséssans contexte (reply)

In [ ]:
faire la pred sur le sommaire horaire par user et par  crypto?

In [ ]:
model positive neut neg puissant
https://huggingface.co/oferweintraub/bert-base-finance-sentiment-noisy-search?text=Microsoft+updates+Outlook%2C+Teams%2C+and+PowerPoint+to+be+hybrid+work+ready

autre candidat
https://huggingface.co/nickmuchi/deberta-v3-base-finetuned-finance-text-classification?text=the+USD+has+been+trending+lower
    
    
classif pr comprendre quelle crypto

In [ ]:
label huge volat predicted avec un bert custom

trouver les crypto de predilection de chaque utilisateur / eliminer ceux qui ont trop peu d'info/ faire la classif dxe topic pr filtrer et reeliminer ceux qui parlent trop pr rien par ex

predire plusieurs classes => court terme : positif et long terme : negatif (juste variable : court terme / long terme / moyen terme manuellement)
        
crypto twitter account recommendation algorithm

filtrer par secteur finance

predire avec bert => acheter levier, spot......

faire dashboard backtest stylé : https://huggingface.co/nickmuchi/finbert-tone-finetuned-finance-topic-classification?text=Company+and+Elon+Musk+are+set+for+a+blockbuster+courtroom+battle+over+Musk%E2%80%99s+attempt+to+terminate+his+%2444+billion+acquisition+deal+for+%24TWTR%2C+according+to+Wedbush+analyst+Dan+Ives.

classifieur de qualités d'une personne => non related , futures , ETHn mec chaud mec nul

#pr less comptes bien connus on va chercher a avoir des signaux constant (pearson haut avec y), pr les moins connus, on va évaluer leur capacité a prédire des mouvements tres volatiles 
si utilisation de crypto bert, bien filtrer avant les tweets car le fine tuning a étz effectué sur des tweets uniquement

features comme vues des tweets pas importants car stockes seulmement une fois fini or on veut une estimation des la publication du sentiment créé et sentiment créable

In [ ]:
voir recommandation apres
donner plus dimportance aux scores de tweet de fin  d'heure comparée au début lors du groupy

In [ ]:
reccommandation crypto n,ested fi

In [ ]:
#### finance topic clasisification ####

topics = ["Analyst Update","Fed | Central Banks","Company | Product News","Treasuries | Corporate Debt","Dividend","Earnings","Energy | Oil","Financials","Currencies","General News | Opinion","Gold | Metals | Materials","IPO","Legal | Regulation","M&A | Investments","Macro","Markets","Politics","Personnel Change","Stock Commentary","Stock Movement",]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("nickmuchi/finbert-tone-finetuned-finance-topic-classification")
model = AutoModelForSequenceClassification.from_pretrained("nickmuchi/finbert-tone-finetuned-finance-topic-classification")
model.to(device)  # Enable CUDA

def pred_topic(tweet):
    inputs = tokenizer(tweet, return_tensors="pt")
    outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    return pd.DataFrame(probs.detach().numpy(), columns=topics,index = ['probs']).sort_values(by=['probs'],ascending=False, axis=1)

def get_max_column_names(df):
    return df.apply(lambda x: x.idxmax(), axis=1).reset_index(drop=True)

res = pd.concat([df.iloc[:,1:],get_max_column_names(res)],axis=1)
res.columns = ['Date','User','Tweet','Topic']